In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [37]:
!wget $data -O data_for_churn.csv

--2023-02-06 20:26:13--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: 'data_for_churn.csv'

     0K .......... .......... .......... .......... ..........  5%  180K 5s
    50K .......... .......... .......... .......... .......... 10% 1.83M 3s
   100K .......... .......... .......... .......... .......... 15%  311K 2s
   150K .......... .......... .......... .......... .......... 20%  320K 2s
   200K .......... .......... .......... .......... .......... 26%  286K 2s
   250K .......... .......... .......... .......... .......... 31% 1.83M 2s
   300K .......... .......... .......... .

In [5]:
df = pd.read_csv("data_for_churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')


In [7]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [8]:
tc=pd.to_numeric(df.totalcharges, errors='coerce')

In [9]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

In [10]:
df[tc.isnull()][['customerid', 'totalcharges']]

,customerid,totalcharges
488,4472-lvygi,0.0
753,3115-czmzd,0.0
936,5709-lvoeq,0.0
1082,4367-nuyao,0.0
1340,1371-dwpaz,0.0
3331,7644-omvmy,0.0
3826,3213-vvolg,0.0
4380,2520-sgtta,0.0
5218,2923-arzlg,0.0
6670,4075-wkniu,0.0


In [11]:
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [12]:
df.churn = (df.churn == 'yes').astype(int)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [15]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [16]:
len(df_train), len(df_test), len(df_val)

(4225, 1409, 1409)

In [17]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [18]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [19]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [20]:
y_test

array([0, 0, 0, ..., 0, 0, 1])

In [21]:
df_full_train = df_full_train.reset_index(drop=True)

In [22]:
df_full_train.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [23]:
global_churn_rate = df_full_train.churn.mean().round(2)
global_churn_rate

0.27

In [24]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [25]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 
               'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 
               'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling','paymentmethod']

In [26]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [27]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [28]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [29]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [30]:
global_churn_rate = df_full_train.churn.mean()
global_churn_rate

0.26996805111821087

In [31]:
df_full_train.partner.value_counts()

no     2932
yes    2702
Name: partner, dtype: int64

In [32]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

0.20503330866025166

In [33]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

0.3298090040927694

In [34]:
global_churn_rate - churn_partner

0.06493474245795922

In [35]:
churn_partner/global_churn_rate

0.7594724924338315

In [36]:
churn_no_partner/global_churn_rate

1.2216593879412643

In [37]:
from IPython.display import display

In [38]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn_rate
    df_group['risk'] = df_group['mean'] / global_churn_rate
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


In [39]:
from sklearn.metrics import mutual_info_score

In [40]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [41]:
mutual_info_score(df_full_train.churn, df_full_train.gender)

0.0001174846211139946

In [42]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)


In [43]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

In [44]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [45]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

0.5953420669577875

In [46]:
df_full_train[df_full_train.tenure > 2].churn.mean()

0.22478269658378816

In [47]:
from sklearn.feature_extraction import DictVectorizer

In [48]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [49]:
dv = DictVectorizer(sparse=False)

In [50]:
X_train = dv.fit_transform(train_dicts)

In [51]:
X_train.shape

(4225, 45)

In [52]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [53]:
X_val = dv.transform(val_dicts)

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:
model = LogisticRegression()
model.fit(X_train, y_train)

c:\Users\OLAMIDE\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [56]:
model.intercept_[0]

-0.10918840348733709

In [57]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.082,
       -0.034, -0.073, -0.336,  0.317, -0.089,  0.004, -0.258,  0.142,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.285, -0.232,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.071, -0.059,  0.142,
       -0.249,  0.216, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.214, -0.089, -0.232, -0.07 ,  0.   ])

In [61]:
y_pred = model.predict_proba(X_val)[:, 1]

In [187]:
churn_decision = (y_pred >= 0.5)

In [66]:
df_val[churn_decision].customerid.head()

3     8433-wxgna
8     3440-jpscl
11    2637-fkfsy
12    7228-omtpn
19    6711-fldfb
Name: customerid, dtype: object

In [67]:
y_val

array([0, 0, 0, ..., 0, 1, 1])

In [69]:
churn_decision.astype(int)

array([0, 0, 0, ..., 0, 1, 1])

In [70]:
y_val == churn_decision

array([ True,  True,  True, ...,  True,  True,  True])

In [166]:
(y_val == churn_decision).mean()

0.8026969481902059

In [167]:
df_pred = pd.DataFrame()
df_pred['customerid'] = df_val.customerid
df_pred['prediction'] = churn_decision.astype(int)
df_pred['actual'] = y_val
df_pred['probability'] = y_pred

In [168]:
df_pred.head()

,customerid,prediction,actual,probability
0,5846-neqvz,0,0,0.008966
1,3645-deygf,0,0,0.204205
2,3590-tcxtb,0,0,0.211922
3,8433-wxgna,1,1,0.543291
4,2654-vbvpb,0,0,0.213495


In [169]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [170]:
df_pred

,customerid,prediction,actual,probability,correct
0,5846-neqvz,0,0,0.008966,True
1,3645-deygf,0,0,0.204205,True
2,3590-tcxtb,0,0,0.211922,True
3,8433-wxgna,1,1,0.543291,True
4,2654-vbvpb,0,0,0.213495,True
...,...,...,...,...,...
1404,0980-pvmrc,0,0,0.313788,True
1405,2325-zusfd,0,1,0.039286,False
1406,7382-dfjtu,0,0,0.136217,True
1407,6521-yytyi,1,1,0.800175,True


In [171]:
df_pred.correct.mean()

0.8026969481902059

In [172]:
dv.get_feature_names()

c:\Users\OLAMIDE\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic

In [173]:
model.coef_[0].round(3)

array([ 0.268, -0.153, -0.238,  0.029, -0.153,  0.066, -0.092, -0.098,
       -0.052, -0.071, -0.298,  0.266, -0.092,  0.004, -0.221,  0.032,
        0.066,  0.089, -0.092, -0.121,  0.232, -0.092, -0.263, -0.269,
        0.145, -0.067, -0.056, -0.086, -0.105,  0.261, -0.193,  0.032,
       -0.155,  0.179, -0.096, -0.092,  0.064, -0.06 , -0.092,  0.028,
        0.248, -0.092, -0.28 , -0.066,  0.   ])

In [174]:
model.intercept_[0]

-0.12361673722455008

In [175]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.268,
 'contract=one_year': -0.153,
 'contract=two_year': -0.238,
 'dependents=no': 0.029,
 'dependents=yes': -0.153,
 'deviceprotection=no': 0.066,
 'deviceprotection=no_internet_service': -0.092,
 'deviceprotection=yes': -0.098,
 'gender=female': -0.052,
 'gender=male': -0.071,
 'internetservice=dsl': -0.298,
 'internetservice=fiber_optic': 0.266,
 'internetservice=no': -0.092,
 'monthlycharges': 0.004,
 'multiplelines=no': -0.221,
 'multiplelines=no_phone_service': 0.032,
 'multiplelines=yes': 0.066,
 'onlinebackup=no': 0.089,
 'onlinebackup=no_internet_service': -0.092,
 'onlinebackup=yes': -0.121,
 'onlinesecurity=no': 0.232,
 'onlinesecurity=no_internet_service': -0.092,
 'onlinesecurity=yes': -0.263,
 'paperlessbilling=no': -0.269,
 'paperlessbilling=yes': 0.145,
 'partner=no': -0.067,
 'partner=yes': -0.056,
 'paymentmethod=bank_transfer_(automatic)': -0.086,
 'paymentmethod=credit_card_(automatic)': -0.105,
 'paymentmethod=electronic_check': 0.261,

In [176]:
small = ['contract', 'tenure', 'monthlycharges']

In [177]:
df_train[small].to_dict(orient='records')

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 99.9},
 {'contract': 'month-to-month', 'tenure': 17, 'monthlycharges': 102.55},
 {'contract': 'two_year', 'tenure': 66, 'monthlycharges': 58.2},
 {'contract': 'month-to-month', 'tenure': 2, 'monthlycharges': 68.95},
 {'contract': 'month-to-month', 't

In [178]:
dicts_train_small = df_train[small].to_dict(orient='records')
dicts_val_small = df_val[small].to_dict(orient='records')

In [179]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [180]:
dv_small.get_feature_names()

c:\Users\OLAMIDE\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'monthlycharges',
 'tenure']

In [181]:
X_train_small = dv_small.transform(dicts_train_small)

In [182]:
X_train_small.shape[0]

4225

In [183]:
y_train.shape[0]

4225

In [184]:
assert X_train_small.shape[0] == y_train.shape[0]

In [185]:
model_small = LogisticRegression()  # default solver is okay
model_small.fit(X_train_small, y_train) # train the model

LogisticRegression()

In [186]:
model_small = LogisticRegression()
model_small.fit(X_train_small, y_train)

LogisticRegression()

In [147]:
w0 = model_small.intercept_[0]
w0

-2.476775657399558

In [148]:
w = model_small.coef_[0].round(3)
w

array([ 0.97 , -0.025, -0.949,  0.027, -0.036])

In [149]:
dict(zip(dv_small.get_feature_names(), w.round(3)))

c:\Users\OLAMIDE\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'contract=month-to-month': 0.97,
 'contract=one_year': -0.025,
 'contract=two_year': -0.949,
 'monthlycharges': 0.027,
 'tenure': -0.036}

Using The Model

In [150]:
dicts_train_full = df_full_train[categorical + numerical].to_dict(orient='records')

In [156]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_train_full)
y_full_train = df_full_train.churn.values

In [157]:
model = LogisticRegression()
model.fit(X_full_train, y_full_train)

LogisticRegression()

In [158]:
dicts_test = df_test[categorical + numerical].to_dict(orient='records')

In [159]:
X_test = dv.transform(dicts_test)

In [160]:
y_pred = model.predict_proba(X_test)[:, 1]

array([0.06224296, 0.17473875, 0.37026701, ..., 0.00638005, 0.16576097,
       0.59688521])

In [197]:
churn_decision = (y_pred >= 0.5)

In [198]:
(churn_decision == y_test).mean()

0.64513839602555

In [207]:
customer = dicts_test[-1]
customer

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 17,
 'monthlycharges': 104.2,
 'totalcharges': 1743.5}

In [208]:
X_small = dv.transform([customer])

In [209]:
model.predict_proba(X_small)[0, 1]

0.5968852088270501

In [211]:
y_test[-1]

1